In [1]:
from math import log, sqrt

import pandas as pd

import numpy as np

from sklearn import linear_model

In [2]:
dtype_dict = {
    'bathrooms': float,
    'bedrooms': float,
    'condition': int,
    'date': str,
    'floors': float,
    'grade': int,
    'id': str,
    'lat': float,
    'long': float,
    'price': float,
    'sqft_above': int,
    'sqft_basement': int,
    'sqft_living': float,
    'sqft_living15': float,
    'sqft_lot': int,
    'sqft_lot15': float,
    'view': int,
    'waterfront': int,
    'yr_built': int,
    'yr_renovated': int,
    'zipcode': str
}

In [3]:
def get_numpy_data(data, features, output):
    data['constant'] = 1 
    return (np.array(data[['constant'] + features]), np.array(data[output],))


def predict_outcome(features, weights):
    '''
    features   N * f
    weights    f * 1
    '''
    return np.dot(features, weights)

In [4]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    return (features / norms, norms)

In [5]:
kc_house_data = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [6]:
(features, output) = get_numpy_data(kc_house_data, ['sqft_living', 'bedrooms'], 'price')

In [7]:
features

array([[  1.00000000e+00,   1.18000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   2.57000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   7.70000000e+02,   2.00000000e+00],
       ..., 
       [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00],
       [  1.00000000e+00,   1.60000000e+03,   3.00000000e+00],
       [  1.00000000e+00,   1.02000000e+03,   2.00000000e+00]])

In [8]:
(feature_normed, norms) = normalize_features(features)

In [9]:
initial_weights = [1., 4., 1.]

In [10]:
initial_prediction = predict_outcome(feature_normed, initial_weights)

In [11]:
def compute_rss(predictions, real):
    errors = real - predictions
    rss = sum(errors * errors)
    return rss

In [12]:
compute_rss(initial_prediction, output)

9217324114222372.0

In [13]:
residual = output - initial_prediction

ro = [0.] * 3
for i in range(len(initial_weights)):
    ro[i] = np.sum(feature_normed[:,i] * (residual + initial_weights[i] * feature_normed[:,i]))

In [14]:
print(ro)

[79400300.014522895, 87939470.823251754, 80966698.66623947]


In [15]:
print("For feature1 l1_penalty=%f" % (ro[1] * 2))
print("For feature2 l1_penalty=%f" % (ro[2] * 2))

For feature1 l1_penalty=175878941.646504
For feature2 l1_penalty=161933397.332479


In [16]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_outcome(feature_matrix, weights)
    
    residual = output - prediction

    ro_i = np.sum(feature_matrix[:,i] * (residual + weights[i] * feature_matrix[:,i]))
    
    if i == 0:
        new_weight_i = ro_i
    elif ro_i < -l1_penalty / 2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty / 2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [17]:
print(lasso_coordinate_descent_step(1,
                                    np.array([[3./sqrt(13), 1./sqrt(10)], [2./sqrt(13), 3./sqrt(10)]]),
                                    np.array([1., 1.]),
                                    np.array([1., 4.]),
                                    0.1))

0.425558846691


In [18]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights.copy()

    while True:
        new_weights = weights.copy()
        for i in range(len(weights)):
            new_weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, new_weights, l1_penalty)

        chg = np.max(np.abs(new_weights - weights))

        if chg < tolerance:
            break

        weights = new_weights

    return weights

In [19]:
initial_weights = np.zeros(3)

l1_penalty = 1e7

tolerance = 1.0

In [20]:
simple_weights = lasso_cyclical_coordinate_descent(feature_normed, output, initial_weights, l1_penalty, tolerance)

In [21]:
compute_rss(predict_outcome(feature_normed, simple_weights), output)

1630492484578352.8

In [22]:
names = ['intercept', 'sqft_living', 'bedrooms']
print(list(zip(names, simple_weights)))

[('intercept', 21624998.819060422), ('sqft_living', 63157246.421593979), ('bedrooms', 0.0)]


## Evaluating LASSO fit with more features

In [23]:
features = [
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated'
]

In [24]:
train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test  = pd.read_csv('kc_house_test_data.csv' , dtype=dtype_dict)

In [25]:
(features_more, output) = get_numpy_data(train, features, 'price')

In [26]:
(feature_more_normed, norms) = normalize_features(features_more)

In [27]:
initial_weights = np.zeros(len(features) + 1)

l1_penalty = 1e7

tolerance = 1.0

In [28]:
weights1e7 = lasso_cyclical_coordinate_descent(feature_more_normed, output, initial_weights, l1_penalty, tolerance)

In [29]:
weights1e7

array([ 24429601.08756559,         0.        ,         0.        ,
        48389173.96813957,         0.        ,         0.        ,
         3317511.21380402,   7329961.8643644 ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [30]:
for i in list(zip(['intercept'] + features, weights1e7)):
    print(i)

('intercept', 24429601.087565593)
('bedrooms', 0.0)
('bathrooms', 0.0)
('sqft_living', 48389173.968139574)
('sqft_lot', 0.0)
('floors', 0.0)
('waterfront', 3317511.2138040178)
('view', 7329961.8643644005)
('condition', 0.0)
('grade', 0.0)
('sqft_above', 0.0)
('sqft_basement', 0.0)
('yr_built', 0.0)
('yr_renovated', 0.0)


In [31]:
l1_penalty = 1e8
weights1e8 = lasso_cyclical_coordinate_descent(feature_more_normed, output, initial_weights, l1_penalty, tolerance)

In [32]:
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(feature_more_normed, output, initial_weights, l1_penalty, tolerance)

In [33]:
for i in list(zip(['intercept'] + features, weights1e8)):
    print(i)

('intercept', 71114625.714887023)
('bedrooms', 0.0)
('bathrooms', 0.0)
('sqft_living', 0.0)
('sqft_lot', 0.0)
('floors', 0.0)
('waterfront', 0.0)
('view', 0.0)
('condition', 0.0)
('grade', 0.0)
('sqft_above', 0.0)
('sqft_basement', 0.0)
('yr_built', 0.0)
('yr_renovated', 0.0)


In [34]:
for i in list(zip(['intercept'] + features, weights1e4)):
    print(i)

('intercept', 78481482.153042942)
('bedrooms', -22092517.827301651)
('bathrooms', 12915269.12473079)
('sqft_living', 93310202.707815617)
('sqft_lot', -2013964.8159875742)
('floors', -4272322.9845620245)
('waterfront', 6482388.3285456877)
('view', 7133738.6578245172)
('condition', 4877806.5960802939)
('grade', 13985275.16528072)
('sqft_above', -15315597.948484594)
('sqft_basement', -5048567.0430999519)
('yr_built', -84043787.07716085)
('yr_renovated', 2822342.4768613358)


## Rescaling learned weights

In [35]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e4 = weights1e4 / norms
normalized_weights1e8 = weights1e8 / norms

In [36]:
print(normalized_weights1e7[3])

161.317454968


## Evaluating each of the learned models on the test data

In [39]:
(features_test, output_tset) = get_numpy_data(test, features, 'price')

print(compute_rss(predict_outcome(features_test, normalized_weights1e7), output_tset))
print(compute_rss(predict_outcome(features_test, normalized_weights1e4), output_tset))
print(compute_rss(predict_outcome(features_test, normalized_weights1e8), output_tset))

2.75962077477e+14
2.28643418822e+14
5.37166151497e+14
